In [1]:
# from kafka import KafkaConsumer
import json
# from json import loads
from kafka import KafkaConsumer
from json import loads
from datetime import datetime
import psycopg2

In [2]:
consumer = KafkaConsumer('jsonAmit',bootstrap_servers=["localhost:9092"],api_version=(0, 10, 1))

In [3]:
# consumer = KafkaConsumer(
#     'jsonAmit',
#      bootstrap_servers=['localhost:9092'],
#      auto_offset_reset='earliest',
#      enable_auto_commit=True,
# #    group_id='my-group',
#      value_deserializer=lambda x: loads(x.decode('utf-8')))

In [4]:
conn = psycopg2.connect( database="oslo_city_bike", user='postgres', password='root', host='127.0.0.1', port= '5432')
cursor = conn.cursor()

In [ ]:
for message in consumer:
    message = message.value
    m = json.loads(message)
    timestamp = m['last_updated']
    dt_object = datetime.fromtimestamp(timestamp)
    m['Date'] = dt_object.strftime("%b %d %Y %H:%M:%S")
    datehrs = dt_object.strftime("%b %d %Y %H:%M:%S")
    for station in m['data']['stations']:
        cursor.execute("INSERT INTO Station_Status (Station_id,is_installed, is_renting, is_returning,last_reported,num_bikes_available,num_docks_available) VALUES (%s,%s,%s,%s,%s,%s,%s)", (station['station_id'],station['is_installed'],station['is_renting'],station['is_returning'],station['last_reported'],station['num_bikes_available'],station['num_docks_available']))
        conn.commit()
#     print('Data at {} added to POSTGRESQL'.format(collection))

conn.close()

In [ ]:
# for message in consumer:
#     print(message.value)